In [1]:
%config Completer.use_jedi = False

import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
from helper import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']

In [2]:
def discriminator(data_dim=186, beat_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    D_in = Input(shape=[data_dim,1])
    inp1 = Concatenate()([D_in, in_label])

    x = Conv1D(filters=48, kernel_size=19, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=64, kernel_size=15, padding='same', strides=3, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=80, kernel_size=11, padding='same', strides=2, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=96, kernel_size=9, padding='same', strides=2, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Conv1D(filters=112, kernel_size=7, padding='same', strides=2, kernel_initializer='he_normal')(x)
    # x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = GlobalAveragePooling1D()(x)

    x1 = Conv1D(filters=48, kernel_size=9, padding='same', strides=4, kernel_initializer='he_normal')(inp1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=64, kernel_size=7, padding='same', strides=3, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=80, kernel_size=5, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=96, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = Conv1D(filters=112, kernel_size=3, padding='same', strides=2, kernel_initializer='he_normal')(x1)
    # x1 = BatchNormalization()(x1)
    x1 = LeakyReLU(alpha=0.2)(x1)
    x1 = GlobalAveragePooling1D()(x1)

    xx = concatenate([x,x1])

    xx = Dense(100)(xx)
    xx = Dense(100)(xx)

    out = Dense(1, activation='sigmoid')(xx)

    model = Model(inputs=[D_in, in_label], outputs=out)
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
    return model
    # model.summary()

# d_model = discriminator(data_dim=186, beat_dim=186)
# plot_model(d_model, to_file='temp.png', show_shapes=True)

In [3]:
# def discriminator(data_dim, beat_dim=186):
    
#     in_label = Input(shape=(beat_dim,1))
#     D_in = Input(shape=[data_dim,1])
#     x = Concatenate()([D_in, in_label])
    
#     x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.2)(x)
    
#     x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.2)(x)
    
#     x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.2)(x)
    
#     x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.2)(x)
    
#     x = Conv1D(filters=32*16, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
#     x = LeakyReLU(alpha=0.2)(x)
    
#     x = Flatten()(x)
#     out = Dense(1, activation='sigmoid')(x)
    
#     model = Model(inputs=[D_in, in_label], outputs=out)
#     opt = Adam(lr=0.0002, beta_1=0.5)
#     loss = 'binary_crossentropy'
#     model.compile(loss=loss, optimizer=opt, metrics=['accuracy'])
#     return model

# d_model = discriminator(data_dim=186, beat_dim=186)
# plot_model(d_model, to_file='temp.png', show_shapes=True)

In [4]:
def generator(noise_dim=186, beat_dim=186, out_dim=186):
    
    in_label = Input(shape=(beat_dim,1))
    G_in = Input(shape=[noise_dim,1])
    x = Concatenate()([G_in, in_label])
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*16, kernel_size=2, strides=2, padding='valid', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)

    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*8, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*4, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32*2, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=32, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
#     x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
#     x = Activation('relu')(x)
    
    x = UpSampling1D()(x)
    x = Conv1D(filters=1, kernel_size=4, strides=2, padding='same', kernel_initializer='he_normal')(x)
    out = Activation('tanh')(x)
    model = Model(inputs=[G_in, in_label], outputs=out)

    return model

# g_model = generator(noise_dim=186, beat_dim=186, out_dim=186)
# plot_model(g_model, to_file='temp.png', show_shapes=True)

In [5]:
def create_gan(d_model, g_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_beat = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_beat])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_beat], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    loss = 'binary_crossentropy'
    model.compile(loss=loss, optimizer=opt)
    return model

# gan_model = create_gan(d_model, g_model)
# plot_model(gan_model, to_file='Final/gan.pdf', show_shapes=True)

In [6]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [7]:
def load_real_samples():
    X = np.load('Data/ForGAN/X.npy')
    y = np.load('Data/ForGAN/y.npy')

    # print (X.shape, y.shape)

    X_N = X[y==0]
    X_S = X[y==1]
    X_V = X[y==2]

    y_N = y[y==0]
    y_S = y[y==1]
    y_V = y[y==2]

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)

#     X_N=X_N.reshape(X_N.shape[0],X_N.shape[1],1)
#     X_S=X_S.reshape(X_S.shape[0],X_S.shape[1],1)
#     X_V=X_V.reshape(X_V.shape[0],X_V.shape[1],1)

    # print (X_N.shape, y_N.shape)
    # print (X_S.shape, y_S.shape)
    # print (X_V.shape, y_V.shape)
    return reshape(X_N), y_N, reshape(X_S), y_S, reshape(X_V), y_V

In [8]:
def generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_samples):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], int(n_samples/3))
    i_S = randint(0, y_S.shape[0], int(n_samples/3))
    i_V = randint(0, y_V.shape[0], int(n_samples/3))
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    labels = keras.utils.to_categorical(np.hstack((y_N[i_N], y_S[i_S], y_V[i_V])))
    # print (labels.shape)
    
    # generate class labels
    y = reshape(np.random.uniform(0.8, 1, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.ones((n_samples, 1))
    return [X, labels], y

In [9]:
# generate points in latent space as input for the generator
# normal noise
def generate_latent_points(latent_dim, n_samples, n_classes=3):
    # generate points in the latent space
#     X_fake = np.random.uniform(0, 1.0, size=[n_samples, latent_dim])
    X_fake = np.random.normal(0,1.0,(n_samples,latent_dim))
    # generate labels
    labels_fake = np.hstack((np.zeros(int(n_samples/3)), np.ones(int(n_samples/3)), 2*np.ones(int(n_samples/3))))
    np.random.shuffle(labels_fake)
    return [reshape(X_fake), keras.utils.to_categorical(labels_fake)]

In [10]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    ecgs = generator.predict([z_input, z_input])
    # create class labels
    y = reshape(np.random.uniform(0, 0.2, n_samples))
#     y = y.reshape(y.shape[0], 1)
#     y = np.zeros((n_samples, 1))
    return [ecgs, labels_input], y

In [11]:
# create and save a plot of generated images
def save_plot(X, n):
    plt.figure(figsize=(10,3))
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(X[i, :, 0])
    plt.show()
    plt.close()

In [12]:
def get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V):
    
    # choose random instances
    i_N = randint(0, y_N.shape[0], 1)
    i_S = randint(0, y_S.shape[0], 1)
    i_V = randint(0, y_V.shape[0], 1)
    
    # select ECG and labels
    X = np.vstack((X_N[i_N], X_S[i_S], X_V[i_V]))
    return X

In [13]:
def save_new_plot(X_R, z_input, n_batch, name):
    n = 3
    Win = (n_batch//3)
    XX = np.vstack((X_R, z_input[0:n,:,:], z_input[Win:Win+n,:,:], z_input[2*Win:2*Win+n,:,:]))
    plt.figure(figsize=(15,5))
    for i in range(n):
        # subplot(R, C, Plot_No)
        plt.subplot(n+1, n, 1 + i)
        plt.axis('off')
        plt.plot(XX[i,:,:])
    for i in range(n, ((n+1)*(n+1))-(n+1)):
        # define subplot
        plt.subplot(n+1, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.plot(XX[i,:,:])
    # plt.show()
    plt.savefig(name, dpi=75)
    plt.close()

In [14]:
# size of the latent space
latent_dim = 186
# size of the data
data = 186
# classes
classes = 3

n_epochs=10

# multiples of three (three classes) (less thyan 24000)
n_batch=300

# Loss Values
G_L = np.infty

# create the discriminator
d_model = discriminator(data_dim=data)
# d_model = discriminator(data_dim=data, input_classes=classes)

# create the generator
g_model = generator(noise_dim=data, beat_dim=data, out_dim=data)
# g_model = generator(noise_dim=latent_dim, input_classes=classes, out_dim=data)

# create the gan
gan_model = create_gan(d_model, g_model)

folder_name = 'Final/Beat_Input'
if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

plot_model(d_model, to_file=folder_name+'disc.pdf', show_shapes=True)
plot_model(g_model, to_file=folder_name+'gen.pdf', show_shapes=True)
plot_model(gan_model, to_file=folder_name+'gan.pdf', show_shapes=True)


# load image data
X_N, y_N, X_S, y_S, X_V, y_V = load_real_samples()
# # train model
# train(g_model, d_model, gan_model, dataset, latent_dim)

bat_per_epo = int(y_S.shape[0] / n_batch)
half_batch = int(n_batch / 2)

plt.ioff()


    
filename = folder_name + 'Plots'
if not os.path.isdir(filename):
    os.mkdir(filename)

model_name = folder_name + 'Model/'
if not os.path.isdir(model_name):
    os.mkdir(model_name)

f = open(folder_name + 'Loss.csv', 'w')
f.write('d_loss1, d_loss2, g_loss \n')
f.close()

f = open(folder_name + 'Stats.csv', 'w')
for i in range(3):
    for mtc in metric_to_calculate:
        f.write(str(mtc)+'_'+str(i)+',')
f.write('\n')
f.close()

# manually enumerate epochs
for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(bat_per_epo):
        
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, half_batch)
        # print (X_real.shape, labels_real.shape, y_real.shape)
        # update discriminator model weights
        d_loss1, _ = d_model.train_on_batch([X_real, X_real], y_real)
        
        # generate 'fake' examples
        [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # print (X_fake.shape, labels.shape, y_fake.shape)
        # update discriminator model weights
        d_loss2, _ = d_model.train_on_batch([X_fake, X_fake], y_fake)
        
        # prepare points in latent space as input for the generator
        [z_input, _] = generate_latent_points(latent_dim, n_batch)
        [X, _], _ = generate_real_samples(X_N, y_N, X_S, y_S, X_V, y_V, n_batch)
        # print (z_input.shape)
        # create inverted labels for the fake samples
        y_gan = reshape(np.random.uniform(0.8, 1, n_batch))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch([z_input, X], y_gan)
        
        if g_loss < G_L:
            G_L = g_loss
            g_model.save(model_name + str(i*1000 + j) + '_cgan_generator.h5')

        f = open(folder_name + 'Loss.csv', 'a')
        f.write(str(d_loss1)+','+str(d_loss2)+','+str(g_loss)+'\n')
        f.close()

        if (j+1)%2 == 0:
            print('>%d, %d/%d, d1=%.5f, d2=%.5f g=%.5f' %(i, j, bat_per_epo, d_loss1, d_loss2, g_loss))
            name = filename+'/'+str(i*1000 + j)+'.jpg'
            # generate ECGs
            z_input  = g_model.predict([z_input, X])
            X_R = get_real_samples(X_N, y_N, X_S, y_S, X_V, y_V)
            save_new_plot(X_R, z_input, n_batch, name)
            
        if (j+1)%10 == 0:
            evaluate(X, z_input, classes, metric_to_calculate, n_batch, folder_name, samples=4)

>0, 1/80, d1=0.39617, d2=1.80103 g=0.40359
>0, 3/80, d1=0.79925, d2=0.50633 g=1.29651
>0, 5/80, d1=0.96453, d2=0.51815 g=1.04163
>0, 7/80, d1=0.77493, d2=0.49567 g=0.80952
>0, 9/80, d1=0.67120, d2=0.55769 g=0.87007
>0, 11/80, d1=0.65577, d2=0.62309 g=0.84715
>0, 13/80, d1=0.68882, d2=0.44463 g=0.82978
>0, 15/80, d1=0.57219, d2=0.42002 g=0.67001
>0, 17/80, d1=0.47481, d2=0.38789 g=0.52266
>0, 19/80, d1=0.43770, d2=0.33595 g=0.44285
>0, 21/80, d1=0.37830, d2=0.34881 g=0.37832
>0, 23/80, d1=0.36960, d2=0.33832 g=0.36659
>0, 25/80, d1=0.36433, d2=0.34456 g=0.34669
>0, 27/80, d1=0.37553, d2=0.35140 g=0.35580
>0, 29/80, d1=0.37154, d2=0.32324 g=0.33796
>0, 31/80, d1=0.34247, d2=0.35116 g=0.33996
>0, 33/80, d1=0.33684, d2=0.34329 g=0.34379
>0, 35/80, d1=0.34480, d2=0.34291 g=0.34032
>0, 37/80, d1=0.36926, d2=0.33459 g=0.33832
>0, 39/80, d1=0.35686, d2=0.31628 g=0.34023
>0, 41/80, d1=0.34208, d2=0.32282 g=0.34540
>0, 43/80, d1=0.33676, d2=0.33466 g=0.33529
>0, 45/80, d1=0.34283, d2=0.34480 g=0

>4, 55/80, d1=0.30711, d2=0.33212 g=0.32105
>4, 57/80, d1=0.33468, d2=0.33069 g=0.32299
>4, 59/80, d1=0.31869, d2=0.33558 g=0.32559
>4, 61/80, d1=0.32586, d2=0.33150 g=0.32777
>4, 63/80, d1=0.32289, d2=0.31868 g=0.32588
>4, 65/80, d1=0.32205, d2=0.31698 g=0.33497
>4, 67/80, d1=0.32840, d2=0.32026 g=0.32305
>4, 69/80, d1=0.33016, d2=0.31872 g=0.32794
>4, 71/80, d1=0.32014, d2=0.33094 g=0.32786
>4, 73/80, d1=0.32489, d2=0.30590 g=0.32930
>4, 75/80, d1=0.32495, d2=0.33063 g=0.33700
>4, 77/80, d1=0.32848, d2=0.33454 g=0.32572
>4, 79/80, d1=0.34018, d2=0.31259 g=0.32373
>5, 1/80, d1=0.32814, d2=0.33207 g=0.32661
>5, 3/80, d1=0.30575, d2=0.32387 g=0.32156
>5, 5/80, d1=0.32182, d2=0.32709 g=0.32960
>5, 7/80, d1=0.31861, d2=0.32436 g=0.32391
>5, 9/80, d1=0.33460, d2=0.31451 g=0.32362
>5, 11/80, d1=0.33368, d2=0.33820 g=0.32579
>5, 13/80, d1=0.32362, d2=0.34163 g=0.32747
>5, 15/80, d1=0.34101, d2=0.32962 g=0.33727
>5, 17/80, d1=0.31443, d2=0.31972 g=0.31785
>5, 19/80, d1=0.30921, d2=0.33795 g=0

>9, 29/80, d1=0.31564, d2=0.32098 g=0.31997
>9, 31/80, d1=0.35881, d2=0.31467 g=0.35433
>9, 33/80, d1=0.36265, d2=0.33288 g=0.33906
>9, 35/80, d1=0.34731, d2=0.32576 g=0.32967
>9, 37/80, d1=0.31903, d2=0.33283 g=0.33310
>9, 39/80, d1=0.34805, d2=0.34111 g=0.32435
>9, 41/80, d1=0.32250, d2=0.32501 g=0.31754
>9, 43/80, d1=0.32880, d2=0.33174 g=0.32907
>9, 45/80, d1=0.34533, d2=0.31895 g=0.33915
>9, 47/80, d1=0.30859, d2=0.32712 g=0.32253
>9, 49/80, d1=0.32254, d2=0.33025 g=0.32200
>9, 51/80, d1=0.32098, d2=0.32731 g=0.34009
>9, 53/80, d1=0.33449, d2=0.33133 g=0.33108
>9, 55/80, d1=0.33878, d2=0.31099 g=0.33277
>9, 57/80, d1=0.33461, d2=0.34959 g=0.31973
>9, 59/80, d1=0.36278, d2=0.31840 g=0.34145
>9, 61/80, d1=0.32712, d2=0.33331 g=0.33476
>9, 63/80, d1=0.33079, d2=0.31813 g=0.31909
>9, 65/80, d1=0.30895, d2=0.33204 g=0.31582
>9, 67/80, d1=0.33408, d2=0.34509 g=0.31965
>9, 69/80, d1=0.33887, d2=0.33683 g=0.33330
>9, 71/80, d1=0.31895, d2=0.32739 g=0.32409
>9, 73/80, d1=0.31398, d2=0.3350